In [9]:
import sys, os
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data_directory = '/Users/user/Projects/allee_effect/data'

def findVals(path, df=None):
    dfStart = 'Results'
    dfDate = ''
    dfTime = ''
    with open(path, 'rt', encoding = 'utf-8', errors='replace') as f:
        if df is None:
            for num, line in enumerate(f, 1):
                if dfStart in line:
                    return num
        for line in f:
            if 'Date' in line:
                dfDate = line.split('\t')[-1].strip('\n')
            if 'Time' in line:
                dfTime = line.split('\t')[-1].strip('\n')
                #dfTime = convert24(dfTime)
                    
    dT = ', '.join((dfDate, dfTime))
    return dT


def addMultiIndex(df, dT):
    tempDf = df.copy()
    tempColumns = tempDf.columns
    newColumns = []
    
    for i in tempColumns:
        if i <= 6:
            newColumns.append((dT, 'MR', i))
        elif i > 6:
            newColumns.append((dT, 'MVF', i))
            
    return newColumns
    

    
def preprocess(data_directory):
    reads_dfs = []
    dateAndTime = []

    for dirName, subdirList, fileList in os.walk(data_directory):
        for fname in fileList:
            extension = fname.split('.')[-1]
            if extension == 'txt':
                path = os.path.join(data_directory, fname)

                df = pd.read_csv(path, skiprows=findVals(path), sep='\t', header=0, index_col=0)
                df.columns = (range(1, df.shape[1]+1))
                df = df.drop(df.columns[-1], axis=1)

                dfDateTime = findVals(path, df)
                dateAndTime.append(dfDateTime)

                #addMultiIndex(df)
                df.columns = pd.MultiIndex.from_tuples(addMultiIndex(df, dfDateTime))
                reads_dfs.append(df)

    odVals = pd.concat(reads_dfs, axis=1)
    
    return odVals, dateAndTime

odVals, dateAndTime = preprocess(data_directory)